CNN(畳み込みニューラルネットワーク)の画像判定学習モデルを用いた、

モルカーと車の画像群からモルカーの画像かどうかを比較するプログラム。

In [ ]:
# GoogleDriveをColabにマウント
from google.colab import drive
drive.mount('/content/drive')

In [10]:
# 画像サイズの指定
img_size = (320, 320) 

# プロジェクトのルートパス
path_root = '/content/drive/MyDrive/molcar_car_compair'

# molcar, car, molcar_carフォルダのパス
path_molcar = path_root + '/molcar'
path_car = path_root + '/car'

path_molcar_car = path_root + '/molcar_car'

# kawaii
mol_aa = '''
      ＿＿＿＿
　 　 ／ /く￣￣ ＼＿
　 　/(二ﾌ　￣￣￣＼ )   < {0}
　｜|    　　 ●⌒●　ヽ
｜    ヽ＿／　　　＼_ |
｜ (⌒)、　　 Ｙ　   ノ
　￣ (＞ー＜  ＞-＜)
　　 　￣　　　　　   ￣
'''

In [ ]:
# GoogleDrive上に必要なフォルダを作成
from pathlib import Path

# GoogleDriveのMyDrive下にmolcar_car_compairフォルダを作成
Path(path_root).mkdir(exist_ok=True)

# 作成したmolcar_car_compairフォルダ下にmolcar, car, molcar_carフォルダを作成
Path(path_molcar).mkdir(exist_ok=True)
Path(path_car).mkdir(exist_ok=True)
Path(path_molcar_car).mkdir(exist_ok=True)

In [ ]:
# クローラーを用いてWeb上から画像収集
!pip install icrawler

# 検索エンジンはBing
from icrawler.builtin import BingImageCrawler

# 「モルカー」で検索した画像を最大100枚保存
crawler = BingImageCrawler(storage={'root_dir': path_molcar})
crawler.crawl(keyword='モルカー', max_num=100)

# 「車」で検索した画像を最大100枚保存
crawler = BingImageCrawler(storage={'root_dir': path_car})
crawler.crawl(keyword='車', max_num=100)

In [ ]:
# 画像データから学習用、テスト用に分けたデータセットであるnpyファイルを作成
import glob
import numpy as np
import cv2
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

img_data = []
label_data = []

def read_dir(img_path, label):
  jpg_files = glob.glob(img_path + "/*.jpg")
  for f in jpg_files:
    img = cv2.imread(f)
    img = cv2.resize(img, img_size)
    img_data.append(img)
    label_data.append(label)

# molcarのlabelは0
read_dir(path_molcar, 0)
# carはのlabelは1
read_dir(path_car, 1)

# img_dataをndarray型へ変換
img_data = np.array(img_data)

# label_dataをOne_Hotベクトルへ変換
one_hot_label_data = np_utils.to_categorical(label_data)
# 整数型（uint8）へ変換
one_hot_label_data = one_hot_label_data.astype('uint8')

# それぞれデータを学習用とテスト用に8:2で分割
x_train, x_test, y_train, y_test = train_test_split(img_data, one_hot_label_data, test_size=0.2)

xy = (x_train, x_test, y_train, y_test)
# npyファイルに保存
np.save(path_root + "/molcar_car.npy", xy)

In [ ]:
# 畳み込みニューラルネットワーク（CNN）を用いて推論モデルをh5ファイルとして作成、保存
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import RMSprop
from keras.utils import np_utils
import keras
import numpy as np
from sklearn.metrics import accuracy_score

# ラベルデータのサイズ
output_size = 2

# One_Hotベクトルデータ形式ではラベル0は[1,0] ラベル1は[0,1]

def load_data_cnn():
  # npyをロードしそれぞれ学習用データ、テスト用データの変数に格納
  x_train, x_test, y_train, y_test = np.load(path_root + "/molcar_car.npy", allow_pickle=True)
  return x_train, y_train, x_test, y_test

def train_cnn(x, y, x_test, y_test):
  model = Sequential()

  # X.shape[1:]が定番、フィルターサイズは奇数、フィルター枚数は問題による2のn乗
  model.add(Conv2D(64,(7,7), padding='same', input_shape=x.shape[1:]))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))#
  model.add(Conv2D(64,(7,7)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.1))

  model.add(Conv2D(64,(7,7), padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Conv2D(64,(7,7)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.45))
  model.add(Dense(output_size))
  model.add(Activation('softmax'))

  # https://keras.io/ja/optimizers/
  # 最適化アルゴリズムはRMSpropを利用
  opt = RMSprop(learning_rate=0.00005, decay=1e-6)

  # https://keras.io/ja/models/sequential/
  # modelをコンパイル
  model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

  #modelに学習データを28個単位で区切って全て学習させる。学習の評価はテストデータを使用する。これを40回繰り返す。
  #batch_sizeは1度に勉強するデータ数、なので280個問題があったら10回問題を解かないと問題集が終わらない、と言ったイメージ
  #epochsは問題集を何回繰り返し解くかを指定、ここでは40回繰り返すように指定
  #validation_data=(x_test,y_test)は、推論モデルの実力は実力テスト（x_testとy_test）で評価する
  # model.fit(x, y, batch_size=28, epochs=40,verbose=1,validation_data=(x_test,y_test))
  model.fit(x, y, batch_size=28, epochs=10,verbose=1,validation_data=(x_test,y_test))

  # HDF5ファイル(.h5)にKerasのモデルを保存。CNNで学習したモデルをHDF5形式でGoogleDriveに保存。以降、AIを利用する際は再度学習の必要なし。
  model.save(path_root + '/molcar_car_cnn_model_conv7_7_pooling.h5')

# npyファイルからデータセットの読み込み
x_train, y_train, x_test, y_test = load_data_cnn()

# モデルの学習
model = train_cnn(x_train, y_train, x_test, y_test)

In [ ]:
# モルカーと車の試験用画像を収集
crawler = BingImageCrawler(storage={'root_dir': path_molcar_car})
# 「モルカー」で検索した画像を最大50枚保存
crawler.crawl(keyword='モルカー', max_num=50)
# 「車」で検索した画像を最大50枚保存
crawler.crawl(keyword='車', max_num=50)

In [14]:
# h5ファイルから学習済みのモデルを呼び出して画像判定
import cv2
import keras
import sys, os, glob
import numpy as np
from keras.models import load_model

# 学習済みのモデルを呼び出す
path_model = path_root + '/molcar_car_cnn_model_conv7_7_pooling.h5'

# ファイル名から画像データを返す関数
def load_image(f):
  img = cv2.imread(f) # 画像集計でnpyファイルを作る際にOpenCV形式だったので、それに合わせる
  img = cv2.resize(img, img_size)
  return img

model = load_model(path_model)

cnt_all = 0
cnt_molcar = 0
cnt_car = 0

jpg_files = glob.glob(path_molcar_car + "/*.jpg")
for jf in jpg_files:
  img = load_image(jf)

  # 新規画像を学習済みモデルに判定させる
  # 画像集計でnpyファイルを作る際にndarray型データだったので、それに合わせる
  prd = model.predict(np.array([img]))

  # print(prd) # 精度を表示する
  prelabel = np.argmax(prd, axis=1) # argmaxはaxis=1なので横方向に見て、パラメータprd(予測結果が[予測1,予測2,予測3]のように入っている)の最大値の場所をを返す。今回で言えばAランクは0、Bランクは1、Cランクは2のように
  if prelabel == 0:
    print(jf.replace(path_root,""), ">>> PUIPUI !   　    　この画像はモルカーです    ", prd)
    cnt_molcar += 1
  elif prelabel == 1:
    print(jf.replace(path_root,""), ">>> NO PUIPUI...  この画像は車です　　　　　       　", prd)
    cnt_car += 1
  cnt_all += 1

print(mol_aa.format('PUIPUI !'))

# 結果出力
print(f'判定した画像総数: {cnt_all}枚')
print(f'モルカーと判定した画像枚数: {cnt_molcar}枚')
print(f'車と判定した画像枚数: {cnt_car}枚')
print(f'モルカーとも車とも判定しなかった画像枚数: {cnt_all - cnt_molcar - cnt_car}枚')

/molcar_car/000001.jpg >>> PUIPUI !   　    　この画像はモルカーです     [[9.999919e-01 8.078167e-06]]
/molcar_car/000002.jpg >>> PUIPUI !   　    　この画像はモルカーです     [[9.9961865e-01 3.8135116e-04]]
/molcar_car/000003.jpg >>> PUIPUI !   　    　この画像はモルカーです     [[0.993996   0.00600396]]
/molcar_car/000004.jpg >>> PUIPUI !   　    　この画像はモルカーです     [[9.999950e-01 4.972759e-06]]
/molcar_car/000005.jpg >>> PUIPUI !   　    　この画像はモルカーです     [[0.9988895  0.00111046]]
/molcar_car/000006.jpg >>> PUIPUI !   　    　この画像はモルカーです     [[9.9998224e-01 1.7748685e-05]]
/molcar_car/000007.jpg >>> PUIPUI !   　    　この画像はモルカーです     [[9.999734e-01 2.656654e-05]]
/molcar_car/000008.jpg >>> PUIPUI !   　    　この画像はモルカーです     [[9.9999738e-01 2.5743711e-06]]
/molcar_car/000009.jpg >>> PUIPUI !   　    　この画像はモルカーです     [[9.999392e-01 6.085209e-05]]
/molcar_car/000010.jpg >>> PUIPUI !   　    　この画像はモルカーです     [[9.9998331e-01 1.6714203e-05]]
/molcar_car/000011.jpg >>> PUIPUI !   　    　この画像はモルカーです     [[9.9963927e-01 3.6072067e-04]]
/molcar_